In [ ]:
import logging
import os
import sys

from cookbook.interface import PluginInstanceRedisInterface
from rmsd_v2.fpocket_client import FPocketClient


logging.basicConfig()
logger = logging.getLogger(__name__)

# Set up redis credentials
redis_host = 'redis'
redis_port = 6379
redis_password = ''
redis_channel = os.environ.get("REDIS_CHANNEL")

# Increase the recursion limit in order to properly serialize Complexes
recursion_limit = 100000
sys.setrecursionlimit(recursion_limit)

plugin_instance = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)
fpocket_client = FPocketClient()

In [ ]:
# Load in example structures

from nanome.api.structure import Complex
from nanome.util import Color
# Load Test Molecules
examples_folder = os.path.join(os.getcwd(), 'examples')
pdb_6nzt = os.path.join(examples_folder, '6nzt.pdb')
pdb_6nzv = os.path.join(examples_folder, '6nzv.pdb')

comp_1 = Complex.io.from_pdb(path=pdb_6nzt)
comp_1.name = "6NZT"
comp_2 = Complex.io.from_pdb(path=pdb_6nzv)
comp_2.name = "6NZV"
comp_list = [comp_1, comp_2]
moving_comp = next(iter([comp in comp_list if comp.name == '6NZT']))
# fixed_comp = next(comp in comp_list if comp.name == '6NZV')

# plugin_instance.update_structures_deep(comp_list)


In [ ]:
ws = plugin_instance.request_workspace()
comp_list = ws.complexes

moving_comp = next(comp for comp in comp_list if comp.name == '6NZT')
fixed_comp = next(comp for comp in comp_list if comp.name == '6NZV')

In [ ]:
# Extract the binding pocket surrounding the ligand.
import copy
from rmsd_v2.RMSDV2 import RMSDV2
from nanome.api import structure
from nanome.util import Color
instance = RMSDV2()

comp = fixed_comp
ligand_name = 'L9J'


def get_binding_site_atoms(comp, ligand_name):
    residue = next(res for res in comp.residues if res.name == ligand_name)
    binding_site_atoms = instance.calculate_binding_site_atoms(comp, residue.atoms)
    return binding_site_atoms


def highlight_atoms(atom_list, on_or_off: bool):
    for atom in atom_list:
        atom.selected = on_or_off
    plugin_instance.update_structures_deep([comp])


def extract_binding_site(comp, binding_site_atoms):
    # Copy comp, and remove all atoms that are not part of the binding site
    new_comp = copy.deepcopy(comp)
    new_comp.name = f'{comp.name} binding site'
    new_comp.index = -1
    new_comp.set_surface_needs_redraw()

    binding_site_indices = [a.index for a in binding_site_atoms]
    print(f"{len(binding_site_indices)} Binding site atoms")
    print(f"original comp has {len(list(new_comp.atoms))} atoms")
    counter = 1
    atom_list = list(new_comp.atoms)
    for i, atom in enumerate(atom_list):
        if not atom.index in binding_site_indices:
            atom.residue.remove_atom(atom)
        else:
            atom.index = -1
    print(f"iterated through {counter} atoms")
    print(f"new comp atom count: {len(list(new_comp.atoms))}")
    return new_comp

fixed_binding_site_atoms = get_binding_site_atoms(fixed_comp, ligand_name)
highlight_atoms(fixed_binding_site_atoms, True)
fixed_binding_site = extract_binding_site(fixed_comp, fixed_binding_site_atoms)
print(f'{len([a for a in fixed_binding_site.atoms if a.selected])} selected Atoms in extracted binding_site')
fixed_binding_site.io.to_pdb(path='fixed_binding_site.pdb')



In [ ]:
# Run FPocket, and highlight the moving structure potential pockets

import tempfile
import time

def cycle_pockets(comp, fpocket_results):
    # fpocket_client = FPocketClient()
    # logger.info(f"Showing pockets for {comp.name}")
    pocket_sets = fpocket_client.parse_results(comp, fpocket_results)
    for pocket in pocket_sets:
        for atom in comp.atoms:
            atom.selected = atom in pocket
        plugin_instance.update_structures_deep([comp])

    # Deselect all atoms
    for atom in comp.atoms:
        atom.selected = False
    plugin_instance.update_structures_deep([comp])

# Run Fpocket to get potential pockets on moving complex, and find best match to fixed complex using SiteMotif
output_dir = '.'
fpocket_results = fpocket_client.run(moving_comp, output_dir)
pocket_folder = os.path.join(fpocket_results, 'pockets')
moving_pocket_pdbs = sorted([f'{pocket_folder}/{fi}' for fi in os.listdir(pocket_folder) if fi.endswith('.pdb')])
cycle_pockets(comp, fpocket_results)


In [ ]:
list_of_pocket_serials = []
for pocket_file in moving_pocket_pdbs:
    pocket_serials = []
    with open(pocket_file) as fd:
        for line in fd:
            if not line.startswith("ATOM"):
                continue
            # Collect serials for pocket atoms
            row = line.split()
            pocket_atom_serial = int(row[1])
            pocket_serials.append(pocket_atom_serial)
    list_of_pocket_serials.append(pocket_serials)
    
# Extract all binding sites
for i, pocket_serial in enumerate(list_of_pocket_serials):
    pocket_site_atoms = [a for a in moving_comp.atoms if a.serial in pocket_serial]
    moving_binding_site = extract_binding_site(moving_comp, pocket_site_atoms)
    print(f"{len(list(moving_binding_site.atoms))} Atoms in binding site complex")
    moving_binding_site.io.to_pdb(path=f'moving_binding{i+1}.pdb')
    # plugin_instance.add_to_workspace([moving_binding_site])
